## Homework 11

1. Explain the idea of bag-of-words model.

2. What are the two methods to treat the meaningless frequently occurring words?

3. Classify the documents in fetch_20newsgroups.

```python
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

news = fetch_20newsgroups(categories=categories, shuffle=True, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(news.data, news.target, test_size = 0.5, random_state=1)
```
4. In two to three paragraphs of prose (i.e. sentences, not bullet lists), summarize and interact with the content that was covered this week in readings and in class meeting. In your summary, you should highlight the major topics, methods, and practices that were covered. Your summary should also interact with the material through personal observations, reflections, and applications to the field of study. In particular, highlight what surprised, enlightened, or otherwise engaged you. In other words, you should think and write critically not just about what was presented but also what you have learned through the session.



Bag-of-words is a model which takes text and shows it as numerical feature vectors which lets us keep a count of how many times the words will appear.

Two methods we can use to treat the meaningless frequently occuring words are term frequency-inverse document frequency which can downweight the frequently occuring words, and stop words removal which will remove very common words such as is or the.

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']
news = fetch_20newsgroups(categories=categories, shuffle=True, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(news.data, news.target, test_size = 0.5, random_state=1)

def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot') if w not in stop]
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)

param_grid = [{
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2']},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, 
                           param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1
                           )
gs_lr_tfidf.fit(X_train, y_train)
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameter set: {'clf__penalty': 'l2', 'vect__tokenizer': <function tokenizer_porter at 0x00000297A2DDD790>} 
CV Accuracy: 0.879
Test Accuracy: 0.885


This week we worked on how to apply machine learning to sentiment analysis. We started with loading IMDb movie reviews which spit the data as positive being 1 and negative being 0. The sentiment value count then is used to see how people felt about the movie. Then we worked on the bag of words model to take the text and turn it into feature vectors. Then there is the frequency-inverse document frequency that we use to deal with very frequent but not useful words. Cleaning the text is done to get rid of extra symbols and filler so we can make it more legible. The words can be tokenized so they can be brought to their base form with lemmatization. Stop words removal then comes in to take out filler words such as and, is, and the. Finally, we can train a logistic model for document classification so see the fits, accuracy, and best parameters.